<a href="https://colab.research.google.com/github/zahraDehghanian97/ddcrp_Cora/blob/master/ddcrp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 23.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910476 sha256=2932a3c4e877f09f28f36c88e65914cb43b5d390d91abbe6cd0649c229f769f5
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch_geometric


In [6]:
from torch_geometric.datasets import Planetoid
import numpy as np
from numpy import seterr, isneginf, array
from scipy.special import gammaln
from scipy.special import logsumexp


dataset = Planetoid("/content/sample_data/Cora" , name = 'Cora')

Processing...
Done!


In [55]:
print(dataset.data.keys)
x = dataset.data.x.detach().cpu().numpy()[0:100]
y = dataset.data.y.detach().cpu().numpy()[0:100]

# x = np.array([[10,0,0,0,0],[10,0,0,0,0],[5,0,0,0,0],[11,0,0,0,1],[0,10,0,0,1],[0,10,0,0,0],[0,0,10,0,0],[0,1,10,0,0],[20,0,2,0,0],[10,0,0,1,0],[10,1,0,10,0],[10,0,2,10,0],[10,0,0,10,0],[10,1,0,1,0],[10,0,0,0,0],])


['x', 'edge_index', 'train_mask', 'test_mask', 'val_mask', 'y']


In [57]:

def dirichlet_likelihood(Xp, hyper):
    if len(Xp.shape) == 2:
        X =sum(Xp)
    else:
        X = Xp
    idx = np.where(X!=0)
    lh = gammaln(len(X)*hyper) + sum(gammaln(X[idx]+hyper))\
    -len(idx)*gammaln(hyper)  - gammaln(sum(X)+len(X) * hyper)
    return lh



hyper = 0.001 #0.2
lhood_fn = lambda x:dirichlet_likelihood(x,hyper)


In [58]:
def window_delay(a,size=1):
    if abs(a) <= size and a >= 0:
        return 1;
    else:
        return 0;



window_size = 20 #20
delay = lambda x:window_delay(x,window_size)


In [59]:
def linear_distance(i,j):
    return i-j


#get the customers linked to customer i
def get_linked(i,link):
    c = []
    q = []
    q.append(i)
    while q:
        cur = q[0]
        c.append(cur)
        for k in range(0,len(link)):
            if (link[k] == cur) and (k not in c) and (k not in q):
                q.append(k)
        q = q[1:]
    return c

def ddcrp_infer(obs,lhood_fn,distance,delay,n_iter,alpha = 0.2):
    n = len(obs)
    cluster = np.array([0]*n)
    link = np.array([0]*n)
    prior = np.random.random(n*n).reshape((n,n))
    #lhood = np.random.random(n)
    merged_lhood = np.random.random(n)
    lhood = list(map(lambda x: lhood_fn(obs[np.where(cluster == x)]) , cluster))  #lhood of each cluster

    obs_lhood = 0 #the likelihood of all obs

    #prior of each customer
    for i in range(0,n):
        for j in range(0,n):
            try:
                if i==j:
                    prior[i][j] = np.log(alpha)
                else:
                    seterr(divide='ignore')
                    prior[i][j] = np.log(delay(distance(i,j)))
                    seterr(divide='warn')
                    prior[i][j][isneginf(prior[i][j])] = 0
            except Exception as e:
                # print(e)
                pass


    for t in range(0,n_iter):
        print("iter "+str(t))
        obs_lhood = 0
        for i in range(0,n):
            #print ("sample"+str(i)+"th:")
            #remove the ith's link
            old_link = link[i]
            old_cluster = cluster[old_link]
            cluster[i] = i
            link[i] = i
            linked = get_linked(i,link)
            # print(linked)
            cluster[linked] = i

            if old_cluster not in linked :
                idx = np.where(cluster == old_cluster)
                lhood[old_cluster] = lhood_fn(obs[idx])
                lhood[i] = lhood_fn(obs[linked])


            #calculate the likelihood of the merged cluster
            for j in np.unique(cluster):

                if j == cluster[i] :
                    merged_lhood[j] = 2*lhood_fn(obs[linked])
                else:
                    merged_lhood[j] = lhood_fn(np.concatenate((obs[linked] , obs[np.where(cluster == j)])))

            log_prob = list(map(lambda x: prior[i][x] + merged_lhood[cluster[x]] - lhood[cluster[x]]-lhood[cluster[i]], np.arange(n)))
            prob = np.exp(log_prob - logsumexp(log_prob))

            #sample z_i
            link[i] = np.random.choice(np.arange(n),1,p=prob)

            #update the likelihood if the link sample merge two cluster
            new_cluster = cluster[link[i]]
            if new_cluster !=i:
                cluster[linked] = new_cluster
                lhood[new_cluster] = merged_lhood[new_cluster]

            #cal the likelihood of all obs
            for u in np.unique(cluster):
                obs_lhood = obs_lhood + lhood[u]

        print("cluster")
        print(cluster)
        print("link")
        print(link)
        print(obs_lhood)

    return cluster,link,obs_lhood

In [60]:

n_iter =20 #200
alpha = 0.0000000000000001  #0.2

distance = linear_distance
cluster,link,lhood = ddcrp_infer(x,lhood_fn,distance,delay,n_iter,alpha)

iter 0
cluster
[ 0  1  1  1  1  1  6  6  6  6  6 11 11 11 11  6 11  6  6 19 19 19 19  6
 24 24 24 24 19 11 30 30 30 30 34 34 34 37 37 37 37 34 42 42 42 30 42 42
 48 48 48 51 51 51 51 51 48 57 57 57 57 51 57 63 63 63 48 57 63 63 70 70
 70 70 70 75 75 75 78 78 78 78 75 75 84 84 84 84 88 88 88 70 92 92 92 92
 78 92 88 99]
link
[ 0  1  1  1  1  3  6  6  7  6  6 11 11 12 13  9 14  6 10 19 19 19 20 18
 24 24 25 26 19 14 30 30 31 32 34 34 35 37 37 38 37 36 42 42 43 33 42 44
 48 48 49 51 51 51 52 53 48 57 57 57 59 54 59 63 63 63 50 57 64 64 70 70
 71 71 72 75 75 75 78 78 78 79 75 82 84 84 85 84 88 88 88 73 92 92 93 94
 80 93 89 99]
-857815.9738621124
iter 1
cluster
[ 0  0  0  3  3  3  6  7  7  7  6 11 12 12 12 11  7  6  6 19 20 19 20 20
 24 25 25 24 20 19 30 31 31 30 34 34 36 37 38 38 37 36 42 43 43 42 42 43
 48 49 49 51 34 51 48 38 34 57 36 59 59 48 43 63 64 63 59 57 49 64 70 71
 70 71 70 75 64 63 78 79 78 79 75 75 84 85 85 84 88 89 88 84 92 93 93 92
 93 85 89 78]
link
[ 0  0  1  3  3  3  6  

diffrent mesurment option for clustering 

such as NMI , purity.

In [66]:
import numpy as np
from sklearn.metrics import accuracy_score

def purity_score(y_true, y_pred):
    y_voted_labels = np.zeros(y_true.shape)
    labels = np.unique(y_true)
    ordered_labels = np.arange(labels.shape[0])
    for k in range(labels.shape[0]):
        y_true[y_true==labels[k]] = ordered_labels[k]
    labels = np.unique(y_true)
    bins = np.concatenate((labels, [np.max(labels)+1]), axis=0)

    for cluster in np.unique(y_pred):
        hist, _ = np.histogram(y_true[y_pred==cluster], bins=bins)
        winner = np.argmax(hist)
        y_voted_labels[y_pred==cluster] = winner
    return accuracy_score(y_true, y_voted_labels)

print(purity_score(y, np.array(cluster)))

0.54


In [67]:
from sklearn.metrics import normalized_mutual_info_score
print(normalized_mutual_info_score(y,cluster))

0.39174693945826905
